In [1]:
from datasets import PartDataset
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
d = PartDataset(root = '../unsupervised3d/shapenetcore_partanno_segmentation_benchmark_v0', classification = True)

{'Earphone': 0, 'Motorbike': 1, 'Rocket': 2, 'Car': 3, 'Laptop': 4, 'Cap': 5, 'Skateboard': 6, 'Lamp': 10, 'Guitar': 8, 'Bag': 9, 'Mug': 7, 'Table': 11, 'Airplane': 12, 'Pistol': 13, 'Chair': 14, 'Knife': 15}


In [3]:
levels = (np.log(2048)/np.log(2)).astype(int)

In [4]:
cutdim = torch.zeros((levels)).long()
#branch = torch.zeros((2048, levels)).long()

In [95]:
def split_ps(point_set):
    #print point_set.size()
    num_points = point_set.size()[0]/2
    diff = point_set.max(dim=0)[0] - point_set.min(dim=0)[0] 
    dim = torch.max(diff, dim = 1)[1][0,0]
    cut = torch.median(point_set[:,dim])[0][0]  
    left_idx = torch.squeeze(torch.nonzero(point_set[:,dim] > cut))
    right_idx = torch.squeeze(torch.nonzero(point_set[:,dim] < cut))
    middle_idx = torch.squeeze(torch.nonzero(point_set[:,dim] == cut))
    
    #if torch.numel(left_idx) > 0:
    #    left_idx = left_idx[:,0]
    #if torch.numel(right_idx) > 0:
    #    right_idx = right_idx[:,0]
    #if torch.numel(middle_idx) > 0:
    #    middle_idx = middle_idx[:,0] 
    
    if torch.numel(left_idx) < num_points:
        left_idx = torch.cat([left_idx, middle_idx[0:1].repeat(num_points - torch.numel(left_idx))], 0)
    if torch.numel(right_idx) < num_points:
        right_idx = torch.cat([right_idx, middle_idx[0:1].repeat(num_points - torch.numel(right_idx))], 0)
    
    left_ps = torch.index_select(point_set, dim = 0, index = left_idx)
    right_ps = torch.index_select(point_set, dim = 0, index = right_idx)
    return left_ps, right_ps, dim 

In [145]:
%%time
for i in range(10):
    #print i
    point_set, class_label = d[i]
    tree = [[] for i in range(levels + 1)]
    cutdim = [[] for i in range(levels)]
    tree[0].append(point_set)
    for level in range(levels):
        for item in tree[level]:
            left_ps, right_ps, dim = split_ps(item)
            tree[level+1].append(left_ps)
            tree[level+1].append(right_ps)
            cutdim[level].append(dim)  
            cutdim[level].append(dim)  
    cutdim = [Variable(torch.from_numpy(np.array(item).astype(np.int64))) for item in cutdim]
    points = torch.stack(tree[-1])
    
    
points_v = Variable(torch.unsqueeze(torch.squeeze(points), 0)).transpose(2,1)

CPU times: user 1.23 s, sys: 12 ms, total: 1.24 s
Wall time: 1.24 s


In [165]:
class KDNet(nn.Module):
    def __init__(self):
        super(KDNet, self).__init__()
        self.conv1 = nn.Conv1d(3,8 * 3,1,1)
        

    def forward(self, x, c):
        x1 = self.conv1(x)
        #x1 = x1.view(-1, 3, 8, 2048)
        #sel = c[-1]
        
        #x1 = torch.index_select(x1, dim = 1, index = sel)
        
        return x1
        
net = KDNet()

In [166]:
x = net(points_v, cutdim)

In [167]:
x

Variable containing:
( 0  ,.,.) = 
 -1.0363e+00 -1.0316e+00 -1.0316e+00  ...  -1.3211e-02 -1.3213e-02 -1.3207e-02
  3.9648e-01  3.9941e-01  3.9941e-01  ...  -7.5476e-01 -7.5476e-01 -7.5477e-01
  3.3743e-01  3.2779e-01  3.2779e-01  ...  -2.3070e-01 -2.3070e-01 -2.3070e-01
                 ...                   ⋱                   ...                
 -8.4430e-01 -8.3342e-01 -8.3342e-01  ...  -1.9406e-01 -1.9406e-01 -1.9405e-01
  2.6654e-02  2.4352e-02  2.4351e-02  ...   1.4608e-01  1.4608e-01  1.4608e-01
 -7.8591e-01 -7.9823e-01 -7.9823e-01  ...   2.9584e-02  2.9581e-02  2.9582e-02
[torch.FloatTensor of size 1x24x2048]